# Лабораторная работа №2

# Решение задач линейного программирования с помощью симплекс метода

### Выполнил: Яковлев Артур, 853501

### Проверила: Костюкова О.И.

#### Вариант 28

Загрузка библиотек, необходимых для решения задачи

In [190]:
import numpy as np
import scipy.linalg as sla

Исходные данные для варианта 28 

In [191]:
dataset1 = (
    np.array([
        [1, 2, -1, 0, 3, 2, 3, 1, 5],
        [1, 0, 0, -1, -1, -3, 0, -6, 1],
        [0, 1, -4, 1, 3, 1, 0, 1, 0],
        [1, 2, -3, 1, -1, 2, 5, 0, 4],  # Матрица А
    ]),
    np.array([-1, -3, 1, -5, 2, -3, 1, -1, -1]),  # вектор с
    np.array([28.0, 59.0, -3.0, 3.0]),  # вектор b (в данной части не используется)
   [0, 3, 4, 7],  # индексы базисного плана
)
initial_plan = np.array([2., 0, 0, 7., 6., 0, 0, 8., 0])  # вектор базисного плана

Реализуем функции для расчета вектора ${\Delta_j}$

In [192]:
def approx(pot, opp_idx, A, c):
    results = [0.0] * A.shape[1]
    all_pos = True
    for i in opp_idx:
        results[i] = pot.dot(A[:, i]) - c[i]
        if results[i] < -1e-9:
            all_pos = False
    return results if not all_pos else None

Далее реализуем сам алгоритм решения. В данной части мы проверяем, существует ли конечное решение задачи, и находим новый базисный план, если это возможно.

In [194]:
def get_next_solution(dataset, B, x, verbose):
    A = dataset[0]
    c = dataset[1]
    b = dataset[2]
    j_b = dataset[3]
    size = A.shape[1]
    j_n = list(i for i in range(size) if i not in j_b)
    A_b = A[:, j_b]
    c_b = c[j_b]
    if B is None:
        B = sla.inv(A_b)
    
    pot = np.dot(c_b, B)    
    approximation = approx(pot, j_n, A, c)
    
    if verbose:
        print("jb: ", j_b)
        print("A_b: ", A_b)
        print("B: ", B)
        print("Step 1")
        print("U: ", pot)
        print("Approx: ", approximation)
    if approximation is None:  # np.all(approximation > -1e-9):
        print(f"{x} is already the most optimized plan")
        return x, 'end'

    j0 = -1
    for i in range(len(approximation)):
        if approximation[i] < -1e-9:
            j0 = i
            break
    
    
    z = B.dot(A[:, j0])
    pos_idx = []
    for i in range(len(z)):
        if z[i] > -1e-9:
            pos_idx.append(i)
    if verbose:
        print("j0: ", j0)
        print("Step 3")
        print("z: ", z)
    
    if len(pos_idx) == 0:
        return None, None
    
    thetas = [x[j_b[i]] / z[i] if z[i] > 1e-9 else np.inf for i in range(len(z))]
    min_value = min(thetas)
    s = thetas.index(min_value)
    new_x = np.zeros(x.shape)
    if verbose:
        print("Step 4")
        for theta in thetas:
            print(f"theta = {theta}")
        print("final theta = ", min_value)
        print("index: ", s)

        print("Step 5")
        print("Обновим план")
        print("Old x: ", x)
    for i in range(len(z)):
        new_x[j_b[i]] = x[j_b[i]] - min_value * z[i]
        if np.abs(new_x[j_b[i]]) < 1e-9:
            new_x[j_b[i]] = 0.0
    new_x[j0] = min_value
    j_b[s] = j0
    if verbose:
        print("New x: ", new_x)
        print("New J_b: ", j_b)
    B = sla.inv(A[:, j_b])
    
    return new_x, B

Теперь с помощью реализации одной итерации, мы можем найти оптимальный план для нашей задачи

In [195]:
def solve(dataset, init_plan, max_iter=100, verbose=True):
    curr_plan = init_plan
    B = None
    for i in range(max_iter):
        if verbose:
            print("\n=====================================\n")
            print(f"Iteration {i}")
        new_plan, B = get_next_solution(dataset, B, curr_plan, verbose=verbose)
        if curr_plan is None:
            print("No solution as function has no upper bound")
            break
        plans_equal = True
        if isinstance(B, str):
            curr_plan = new_plan
            break
        else:
            curr_plan = new_plan
    print(f"\n\nFinal plan: {curr_plan}")
    return curr_plan, dataset[3]

In [196]:
# Before using this, activate part with dataset, otherwise some entries might have wrong values assigned to them
solve(dataset1, initial_plan)



Iteration 0
jb:  [0, 3, 4, 7]
A_b:  [[ 1  0  3  1]
 [ 1 -1 -1 -6]
 [ 0  1  3  1]
 [ 1  1 -1  0]]
B:  [[ 0.55   0.025 -0.4    0.425]
 [-0.45   0.025  0.6    0.425]
 [ 0.1    0.05   0.2   -0.15 ]
 [ 0.15  -0.175 -0.2    0.025]]
Step 1
U:  [ 1.75   0.125 -2.    -2.875]
Approx:  [0.0, -1.2499999999999991, 13.875, 0.0, 0.0, -1.6249999999999973, -10.124999999999998, 0.0, -1.6249999999999982]
j0:  1
Step 3
z:  [1.55 0.55 0.1  0.15]
Step 4
theta = 1.2903225806451613
theta = 12.727272727272732
theta = 60.000000000000014
theta = 53.33333333333333
final theta =  1.2903225806451613
index:  0
Step 5
Обновим план
Old x:  [2. 0. 0. 7. 6. 0. 0. 8. 0.]
New x:  [0.         1.29032258 0.         6.29032258 5.87096774 0.
 0.         7.80645161 0.        ]
New J_b:  [1, 3, 4, 7]


Iteration 1
jb:  [1, 3, 4, 7]
A_b:  [[ 2  0  3  1]
 [ 0 -1 -1 -6]
 [ 1  1  3  1]
 [ 2  1 -1  0]]
B:  [[ 0.35483871  0.01612903 -0.25806452  0.27419355]
 [-0.64516129  0.01612903  0.74193548  0.27419355]
 [ 0.06451613  0.0483871

(array([0.        , 0.        , 0.        , 6.58940397, 6.2384106 ,
        0.        , 0.52980132, 7.69536424, 0.        ]),
 [6, 3, 4, 7])

Рассмотрим решение задачи для другого варианта

In [197]:
dataset2 = (
    np.array([
        [1, 2, -1, 0, 3, 2, 3, 1, 5],
        [1, 0, 0, -1, -1, -3, 0, -6, 1],
        [0, 1, -4, 1, 3, 1, 0, 1, 0],
        [1, 2, -3, 1, -1, 2, 5, 0, 4],  # Матрица А
    ]),
    np.array([-1, -1, -1, -5, 1, -3, 1, -1, -1]),  # вектор с
    np.array([28.0, 59.0, -3.0, 3.0]),  # вектор b (в данной части не используется)
    [0, 1, 2, 7],  # индексы базисного плана
)
initial_plan2 = np.array([2, 4, 2, 0, 0, 0, 0, 4, 0])  # вектор базисного плана

In [198]:
# Update prev sell before launch
solve(dataset2, initial_plan2)



Iteration 0
jb:  [0, 1, 2, 7]
A_b:  [[ 1  2 -1  1]
 [ 1  0  0 -6]
 [ 0  1 -4  1]
 [ 1  2 -3  0]]
B:  [[-10.          -3.          -8.          14.        ]
 [  7.           2.           5.          -9.        ]
 [  1.33333333   0.33333333   0.66666667  -1.66666667]
 [ -1.66666667  -0.66666667  -1.33333333   2.33333333]]
Step 1
U:  [ 3.33333333  1.33333333  3.66666667 -5.66666667]
Approx:  [0.0, 0.0, 0.0, 1.6666666666666647, 24.33333333333335, -2.0000000000000018, -19.333333333333343, 0.0, -3.6666666666666643]
j0:  5
Step 3
z:  [ 9. -5. -1.  2.]
Step 4
theta = 0.22222222222222204
theta = inf
theta = inf
theta = 1.9999999999999991
final theta =  0.22222222222222204
index:  0
Step 5
Обновим план
Old x:  [2 4 2 0 0 0 0 4 0]
New x:  [0.         5.11111111 2.22222222 0.         0.         0.22222222
 0.         3.55555556 0.        ]
New J_b:  [5, 1, 2, 7]


Iteration 1
jb:  [5, 1, 2, 7]
A_b:  [[ 2  2 -1  1]
 [-3  0  0 -6]
 [ 1  1 -4  1]
 [ 2  2 -3  0]]
B:  [[-1.11111111 -0.33333333 -0.888

(array([0.    , 0.    , 1.8375, 0.    , 1.3   , 0.    , 2.1625, 3.45  ,
        0.    ]),
 [6, 4, 2, 7])

**Теперь с помощью данного алгоритма решим задачу оптимизации.**

<img src="task.png">

Составим начальную матрицу $A$, опираясь на состав указанных напитков.

In [199]:
A = np.array([
        [1., 0., 0., 0.7, 0.6, 0.5],
        [0., 1., 0., 0.3, 0. , 0.2],
        [0., 0., 1., 0.,  0.4, 0.3],
])

# столбцы: ябл., вин., клюкв., ябл-вин., ябл-клюкв., фрукт.
# строки: ябл., вин., клюкв.

Из заданных ограничений на начальное количество сока каждого вида, получим вектор $b$.

In [200]:
b = np.array([3000, 1900, 2500])

Найдем прибыль для каждой смеси.

Для яблочно-виноградной смеси прибыль составит: $40 - 0.7 \times 20 - 0.3 \times 23 = 19.1$

Для яблочно-клюквенной смеси: $39 - 0.6 \times 20 - 0.4 \times 18 = 19.8$

Для фруктовой смеси: $42 - 0.5 \times 20 - 0.2 \times 23 - 0.3 \times 18 = 22$

Для яблочного сока: $40 - 20 = 20$

Для виноградного сока: $42 - 23 = 19$

Для клюквенного сока: $37 - 18 = 19$

Поэтому начальный вектор $c$ имеет вид:

In [201]:
c = np.array([20., 19., 19., 19.1, 19.8, 22.])

Остается найти вектор базисного плана

У нашей задачи кроме вектора $b$ есть еще ряд ограничений на базисный план. Известно, что есть заказ на определенные виды напитков, задающий нижнюю границу, а также запрет на производство более 2000 единиц любого товара, что задает верхнюю границу. Запишем данные ограничения в виде двух векторов

In [202]:
lower_bound = np.array([600., 0., 0., 300., 0., 1000.])
upper_bound = np.array([2000., 2000., 2000., 2000., 2000., 2000.])

Далее нам необходимо свести задачу к каноническому виду.

Наша задача имеет вид $c^Tx \rightarrow max, Ax=b, d_* \leq x \leq d^*$

Введем вектор $z=x-d_*$. Получаем эквивалентную задачу вида $c^Tz \rightarrow max, Az=\overline{b}, 0 \leq z \leq \overline{d}$, где $\overline{b} = b - Ad_*, \overline{d}=d^* - d_*$

Полученную задачу можно записать в виде $c^Tz \rightarrow max, Az = \overline{b}, z + y = \overline{d}, z \geq 0, y \geq 0$, из которой, в свою очередь, получаем задачу в каноническом виде:

$$\overline{c}^TX \rightarrow max, \overline{A}X = \overline{b}, X \geq 0,$$

где $X=(z, y)$ - вектор искомых переменных,

$$
\overline{A} = \begin{pmatrix}
A & \mathbb{O}\\
E_n & E_n
\end{pmatrix} \in \mathbb{R}^{(m+n) \times 2n}, \hat{b} = \begin{pmatrix}
\overline{b}\\
\overline{d}
\end{pmatrix}, 
\overline{c}^T = (c^T, 0^T).
$$

Приведем нашу задачу к данному виду.

In [203]:
def convert_to_canon(A, b, c, lower, upper):
    d = upper - lower
    A_new = np.concatenate((np.concatenate((A, np.zeros(A.shape)), axis=1), np.concatenate((np.eye(A.shape[1]), np.eye(A.shape[1])), axis=1)), axis=0)
    b_new = np.concatenate((b - A @ lower, d))
    c_new = np.concatenate((c, np.zeros(c.shape)))
    return {'A': A_new, 'b': b_new, 'c': c_new}

Для нахождения базисного вектора к нашей начальной оптимизационной задачи необходимо применить первую фазу симплекс метода. Она выражается в виде:

$$
-e^Tx \rightarrow \underset{x, x_u}{max}, Ax + Ex_u = b, x \geq 0, x_u \geq 0
$$

В качестве базисного плана для первой фазы подходит вектор $ x = x(J) = 0, x_u = x(J_u) = 0$ с базисом $J_b = J_u$ и базисной матрицей $A_b = E$. Далее, в случае выполнения необходимых условий, по результату первой фазы мы сможем с помощью реализованного ранее алгоритма найти решение исходной задачи.

In [204]:
def find_solution(data, verbose=True):
    j_b = list(range(data['A'].shape[1], data['A'].shape[1] + data['A'].shape[0]))
    c = np.concatenate((np.zeros(data['A'].shape[1]), -np.ones(data['A'].shape[0])), axis=0)
    A = np.concatenate((data['A'], np.eye(data['A'].shape[0])), axis=1)
    x = np.concatenate((np.zeros(data['A'].shape[1]), data['b']), axis=0)
    init_plan, j_b = solve((A, c, b, j_b), x, verbose=verbose)
    if not all(np.abs(x[data['A'].shape[1]:]) > 1e-9):
        print('\nNo valid plan exists\n')
        return None
    if not all(j in range(data['A'].shape[1], data['A'].shape[1] + data['A'].shape[0]) for j in j_b):
        print('\n\nSolution exists, starting iteration process to find it\n\n')
        init_plan = init_plan[:data['A'].shape[1]]
        return solve((data['A'], data['c'], data['b'], j_b), init_plan, verbose=verbose)

Используем данную функцию для решения пункта $a)$ данной задачи

In [205]:
x, _ = find_solution(convert_to_canon(A, b, c, lower_bound, upper_bound))



Iteration 0
jb:  [12, 13, 14, 15, 16, 17, 18, 19, 20]
A_b:  [[1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]]
B:  [[ 1.  0. -0.  0. -0.  0. -0.  0. -0.]
 [ 0.  1. -0.  0. -0.  0. -0.  0. -0.]
 [ 0.  0.  1.  0. -0.  0. -0.  0. -0.]
 [ 0.  0.  0.  1. -0.  0. -0.  0. -0.]
 [ 0.  0.  0.  0.  1.  0. -0.  0. -0.]
 [ 0.  0.  0.  0.  0.  1. -0.  0. -0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0. -0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1. -0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.]]
Step 1
U:  [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
Approx:  [-2.0, -2.0, -2.0, -2.0, -2.0, -2.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
j0:  0
Step 3
z:  [1. 0. 0. 1. 0. 0. 0. 0. 0.]
Step 4
theta = 1690.0
theta = inf
theta = inf
theta = 1400.0
theta = inf
theta = inf
the

Тогда план для исходной задачи примет вид:

In [206]:
x[:c.shape[0]] + lower_bound

array([ 600.        , 1410.        , 1106.66666667,  300.        ,
       1983.33333333, 2000.        ])

Найдем итоговую прибыль

In [207]:
print(c @ (x[:c.shape[0]] + lower_bound))

148816.6666666667


$b)$ Решить задачу в случае, если бы заказ на яблочный и яблочно-виноградный сок отсутствовал

В данном случае изменится лишь вектор $d_*$, отвечающий за нижнее ограничение нашего плана

In [208]:
new_lower_bound = np.array([0., 0., 0., 0., 0., 1000.]) 

Далее решаем аналогично пункту $a)$

В этом и последующем пункте не будет полного вывода процесса выполнения алгоритма, так как все происходит аналогично первому, и количество итераций достаточно большое, что сделает отчет крайне объемным. Это достигается изменением значения параметра $verbose$ нашей функции на $False$.

In [209]:
x, _ = find_solution(convert_to_canon(A, b, c, new_lower_bound, upper_bound), verbose=False)

[   0.         1157.14285714 1100.         1142.85714286 2000.
 1000.         2000.          842.85714286  900.          857.14285714
    0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.
    0.        ] is already the most optimized plan


Final plan: [   0.         1157.14285714 1100.         1142.85714286 2000.
 1000.         2000.          842.85714286  900.          857.14285714
    0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.
    0.        ]


Solution exists, starting iteration process to find it


[ 800. 1500. 1100.    0. 2000. 1000. 1200.  500.  900. 2000.    0.    0.] is already the most optimized plan


Final plan: [ 800. 1500. 1100.    0. 2000. 1000. 1200.  500.  900. 2000.    0.    0.]


В результате получаем план:

In [210]:
x[:c.shape[0]] + new_lower_bound

array([ 800., 1500., 1100.,    0., 2000., 2000.])

Итоговая прибыль равна:

In [211]:
print(c @ (x[:c.shape[0]] + new_lower_bound))

149000.0


$c)$ При возможности закупить 300л сока, какой лучше выбрать для максимизации прибыли?

Для решения рассмотрим все 3 случая. В данной ситуации меняться будет лишь вектор $b$.

**Случай 1**: яблочный сок

In [186]:
b_apple = np.array([3300, 1900, 2500])

In [187]:
x, _ = find_solution(convert_to_canon(A, b_apple, c, lower_bound, upper_bound), verbose=False)

[   0.         1285.71428571 1100.          414.28571429 2000.
 1000.         1400.          714.28571429  900.         1285.71428571
    0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.
    0.        ] is already the most optimized plan


Final plan: [   0.         1285.71428571 1100.          414.28571429 2000.
 1000.         1400.          714.28571429  900.         1285.71428571
    0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.
    0.        ]


Solution exists, starting iteration process to find it


[ 290. 1410. 1100.    0. 2000. 1000. 1110.  590.  900. 1700.    0.    0.] is already the most optimized plan


Final plan: [ 290. 1410. 1100.    0. 2000. 1000. 1110.  590.  900. 1700.    0.    0.]


Полученный план:

In [188]:
x[:c.shape[0]] + lower_bound

array([ 890., 1410., 1100.,  300., 2000., 2000.])

Прибыль:

In [189]:
print(c @ (x[:c.shape[0]] + lower_bound))

154820.0


**Случай 2**: виноградный сок

In [165]:
b_grapes = np.array([3000, 2200, 2500])

In [166]:
x, _ = find_solution(convert_to_canon(A, b_grapes, c, lower_bound, upper_bound), verbose=False)

[   0.         1710.         1106.66666667    0.         1983.33333333
 1000.         1400.          290.          893.33333333 1700.
   16.66666667    0.            0.            0.            0.
    0.            0.            0.            0.            0.
    0.        ] is already the most optimized plan


Final plan: [   0.         1710.         1106.66666667    0.         1983.33333333
 1000.         1400.          290.          893.33333333 1700.
   16.66666667    0.            0.            0.            0.
    0.            0.            0.            0.            0.
    0.        ]


Solution exists, starting iteration process to find it


[   0.         1710.         1106.66666667    0.         1983.33333333
 1000.         1400.          290.          893.33333333 1700.
   16.66666667    0.        ] is already the most optimized plan


Final plan: [   0.         1710.         1106.66666667    0.         1983.33333333
 1000.         1400.          290.          893.33333333

Полученный план:

In [167]:
x[:c.shape[0]] + lower_bound

array([ 600.        , 1710.        , 1106.66666667,  300.        ,
       1983.33333333, 2000.        ])

Прибыль:

In [168]:
print(c @ (x[:c.shape[0]] + lower_bound))

154516.6666666667


**Случай 3:** клюквенный сок

In [180]:
b_cranberries = np.array([3000, 1900, 2800])

In [183]:
x, _ = find_solution(convert_to_canon(A, b_cranberries, c, lower_bound, upper_bound), verbose=False)

[   0.         1410.         1406.66666667    0.         1983.33333333
 1000.         1400.          590.          593.33333333 1700.
   16.66666667    0.            0.            0.            0.
    0.            0.            0.            0.            0.
    0.        ] is already the most optimized plan


Final plan: [   0.         1410.         1406.66666667    0.         1983.33333333
 1000.         1400.          590.          593.33333333 1700.
   16.66666667    0.            0.            0.            0.
    0.            0.            0.            0.            0.
    0.        ]


Solution exists, starting iteration process to find it


[   0.         1410.         1406.66666667    0.         1983.33333333
 1000.         1400.          590.          593.33333333 1700.
   16.66666667    0.        ] is already the most optimized plan


Final plan: [   0.         1410.         1406.66666667    0.         1983.33333333
 1000.         1400.          590.          593.33333333

Полученный план:

In [184]:
x[:c.shape[0]] + lower_bound

array([ 600.        , 1410.        , 1406.66666667,  300.        ,
       1983.33333333, 2000.        ])

Прибыль:

In [185]:
print(c @ (x[:c.shape[0]] + lower_bound))

154516.66666666666


По результатам данных трех экспериментов мы видим, что наибольшая прибыль будет в случае, если мы докупим 300л яюлочного сока.